# Strands Agents with AgentCore Memory (Short-Term Memory) - Using MemoryManager


## Introduction

This tutorial demonstrates how to build a **personal agent** using Strands agents with AgentCore **short-term memory** using **MemoryManager** and **MemorySessionManager**. The agent remembers recent conversations in the session using `get_last_k_turns` and can continue conversations seamlessly when user returns.

**NOTE: This is the Short Term Memory Sample version using the MemoryManager & MemorySessionManager.**


### Tutorial Details

| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Short Term Conversational                                                        |
| Agent type          | Personal Agent                                                                   |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Haiku 4.5                                                      |
| Tutorial components | AgentCore Short-term Memory with MemoryManager, AgentInitializedEvent and MessageAddedEvent hooks   |
| Example complexity  | Beginner                                                                         |

You'll learn to:
- Use short-term memory for conversation continuity with MemoryManager
- Retrieve last K conversation turns using MemorySessionManager
- Web search tool for real-time information
- Initialize agents with conversation history using session management
- Can use this to help Migrate from MemoryClient to MemoryManager architecture

## Architecture
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## Prerequisites

To execute this tutorial you will need:
- Python 3.10+
- AWS credentials with AgentCore Memory permissions
- Amazon Bedrock AgentCore SDK with MemoryManager support
- Access to Amazon Bedrock models

Let's get started by setting up our environment!

## Step 1: Setup and Imports

In [3]:
!pip install -qr requirements.txt

In [4]:
import logging
from datetime import datetime
from botocore.exceptions import ClientError

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("personal-agent")

In [5]:
# Import required modules for Strands Agent
import os
from strands import Agent, tool
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent

# Import memory management modules
from bedrock_agentcore_starter_toolkit.operations.memory.manager import MemoryManager
from bedrock_agentcore.memory.constants import ConversationalMessage, MessageRole
from bedrock_agentcore.memory.session import MemorySession, MemorySessionManager

# Define message role constants
USER = MessageRole.USER
ASSISTANT = MessageRole.ASSISTANT

# Configuration
REGION = os.getenv('AWS_REGION', 'us-east-1') # AWS region for the agent
ACTOR_ID = "user_123" # It can be any unique identifier (AgentID, User ID, etc.)
SESSION_ID = "personal_session_001" # Unique session identifier

# Import boto3 for IAM role creation
import boto3
import json as json_module

## Step 2: Web Search Tool

First, let's create a simple web search tool for the agent. This remains unchanged from the original implementation.

In [6]:
from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

@tool
def websearch(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """Search the web for updated information.
    
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "Rate limit reached. Please try again later."
    except DDGSException as e:
        return f"Search error: {e}"
    except Exception as e:
        return f"Search error: {str(e)}"

logger.info("✅ Web search tool ready")

2025-12-13 18:39:12,188 - INFO - ✅ Web search tool ready


## Step 3: Create Memory Resource using MemoryManager

For short-term memory, we create a memory resource without any strategies using MemoryManager. This stores raw conversation turns that can be retrieved with `get_last_k_turns`.

**NOTE: This section uses the MemoryManager architecture instead of the legacy MemoryClient.**

In [7]:
# Initialize Memory Manager 
memory_manager = MemoryManager(region_name=REGION)
memory_name = "PersonalAgentMemoryManager"

logger.info(f"✅ MemoryManager initialized for region: {REGION}")
logger.info(f"Memory manager type: {type(memory_manager)}")

# Create memory resource using MemoryManager
logger.info(f"Creating memory '{memory_name}' for short-term conversational storage...")

try:
    memory = memory_manager.get_or_create_memory(
        name=memory_name,
        strategies=[],  # No strategies for short-term memory
        description="Short-term memory for personal agent",
        event_expiry_days=7,  # Retention period for short-term memory
        memory_execution_role_arn=None,  # Optional for short-term memory
    )
    memory_id = memory.id
    logger.info(f"✅ Successfully created/retrieved memory with MemoryManager:")
    logger.info(f"   Memory ID: {memory_id}")
    logger.info(f"   Memory Name: {memory.name}")
    logger.info(f"   Memory Status: {memory.status}")
    
except Exception as e:
    # Handle any errors during memory creation with enhanced error reporting
    logger.error(f"❌ Memory creation failed: {e}")
    logger.error(f"Error type: {type(e).__name__}")
    import traceback
    traceback.print_exc()
    
    # Cleanup on error - delete the memory if it was partially created
    if 'memory_id' in locals():
        try:
            logger.info(f"Attempting cleanup of partially created memory: {memory_id}")
            memory_manager.delete_memory(memory_id)
            logger.info(f"✅ Successfully cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.error(f"❌ Failed to clean up memory: {cleanup_error}")
    
    # Re-raise the original exception
    raise

✅ MemoryManager initialized for region: us-west-2
2025-12-13 18:39:32,540 - INFO - ✅ MemoryManager initialized for region: us-west-2
2025-12-13 18:39:32,541 - INFO - ✅ MemoryManager initialized for region: us-west-2
2025-12-13 18:39:32,542 - INFO - Memory manager type: <class 'bedrock_agentcore_starter_toolkit.operations.memory.manager.MemoryManager'>
2025-12-13 18:39:32,542 - INFO - Creating memory 'PersonalAgentMemoryManager' for short-term conversational storage...
Created memory: PersonalAgentMemoryManager-W54AFBBqJZ
2025-12-13 18:39:33,807 - INFO - Created memory: PersonalAgentMemoryManager-W54AFBBqJZ
Created memory PersonalAgentMemoryManager-W54AFBBqJZ, waiting for ACTIVE status...
2025-12-13 18:39:33,808 - INFO - Created memory PersonalAgentMemoryManager-W54AFBBqJZ, waiting for ACTIVE status...
Memory PersonalAgentMemoryManager-W54AFBBqJZ is now ACTIVE (took 172 seconds)
2025-12-13 18:42:26,321 - INFO - Memory PersonalAgentMemoryManager-W54AFBBqJZ is now ACTIVE (took 172 seconds

## Step 4: Initialize Session Manager

This section introduces the MemorySessionManager for session-based memory operations and creates a MemorySession to manage actor & session

In [8]:
# Initialize the session memory manager
session_manager = MemorySessionManager(memory_id=memory.id, region_name=REGION)

# Create a memory session for the specific actor/session combination
user_session = session_manager.create_memory_session(
    actor_id=ACTOR_ID, 
    session_id=SESSION_ID
)

logger.info(f"✅ Session manager initialized for memory: {memory.id}")
logger.info(f"✅ Memory session created for actor: {ACTOR_ID}, session: {SESSION_ID}")
logger.info(f"Session manager type: {type(session_manager)}")
logger.info(f"Memory session type: {type(user_session)}")

2025-12-13 18:42:26,407 - INFO - 💬 Creating new conversation for actor 'user_123' in session 'personal_session_001'...
2025-12-13 18:42:26,408 - INFO - ✅ Session manager initialized for memory: PersonalAgentMemoryManager-W54AFBBqJZ
2025-12-13 18:42:26,409 - INFO - ✅ Memory session created for actor: user_123, session: personal_session_001
2025-12-13 18:42:26,409 - INFO - Session manager type: <class 'bedrock_agentcore.memory.session.MemorySessionManager'>
2025-12-13 18:42:26,409 - INFO - Memory session type: <class 'bedrock_agentcore.memory.session.MemorySession'>


## Step 5: Memory Hook Provider

This step defines our custom `MemoryHookProvider` class that automates memory operations using the MemorySession. Hooks are special functions that run at specific points in an agent's execution lifecycle. The memory hook we're creating serves two primary functions:
1. **To load recent conversation**: We use the `AgentInitializedEvent` hook to automatically load recent conversation history when the agent is initialized.
2. **To store the last message**: Stores new conversational messages using the session manager.

**KEY CHANGES from MemoryClient version:**
- Uses MemorySession instead of MemoryClient
- Uses ConversationalMessage objects instead of tuples
- Uses add_turns() instead of create_event()
- Uses MessageRole enum for type safety

In [9]:
class MemoryHookProvider(HookProvider):
    def __init__(self, memory_session: MemorySession):  # Accept MemorySession instead
        self.memory_session = memory_session
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """Load recent conversation history when agent starts using MemorySession"""
        try:
            # Use the pre-configured memory session (no need for actor_id/session_id)
            recent_turns = self.memory_session.get_last_k_turns(k=5)
            
            if recent_turns:
                # Format conversation history for context
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        # Handle both EventMessage objects and dict formats
                        if hasattr(message, 'role') and hasattr(message, 'content'):
                            role = message['role']
                            content = message['content']
                        else:
                            role = message.get('role', 'unknown')
                            content = message.get('content', {}).get('text', '')
                        context_messages.append(f"{role}: {content}")
                
                context = "\n".join(context_messages)
                # Add context to agent's system prompt
                event.agent.system_prompt += f"\n\nRecent conversation:\n{context}"
                logger.info(f"✅ Loaded {len(recent_turns)} conversation turns using MemorySession")
                
        except Exception as e:
            logger.error(f"Memory load error: {e}")

    def on_message_added(self, event: MessageAddedEvent):
        """Store messages in memory using MemorySession"""
        messages = event.agent.messages
        try:
            if messages and len(messages) > 0 and messages[-1]["content"][0].get("text"):
                message_text = messages[-1]["content"][0]["text"]
                message_role = MessageRole.USER if messages[-1]["role"] == "user" else MessageRole.ASSISTANT
                
                # Use memory session instance (no need to pass actor_id/session_id)
                result = self.memory_session.add_turns(
                    messages=[ConversationalMessage(message_text, message_role)]
                )
                
                event_id = result['eventId']
                logger.info(f"✅ Stored message with Event ID: {event_id}, Role: {message_role.value}")
                
        except Exception as e:
            logger.error(f"Memory save error: {e}")
            import traceback
            logger.error(f"Full traceback: {traceback.format_exc()}")
    
    def register_hooks(self, registry: HookRegistry):
        # Register memory hooks
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)
        logger.info("✅ Memory hooks registered with MemorySession")


## Step 6: Create Personal Agent with Web Search

This agent uses the MemoryHookProvider that works with MemorySession created from MemorySessionManager.

In [10]:
def create_personal_agent():
    """Create personal agent with memory and web search using MemorySession"""
    agent = Agent(
        name="PersonalAssistant",
        model="global.anthropic.claude-haiku-4-5-20251001-v1:0",  # or your preferred model
        system_prompt=f"""You are a helpful personal assistant with web search capabilities.
        
        You can help with:
        - General questions and information lookup
        - Web searches for current information
        - Personal task management
        
        When you need current information, use the websearch function.
        Today's date: {datetime.today().strftime('%Y-%m-%d')}
        Be friendly and professional.""",
        hooks=[MemoryHookProvider(user_session)], 
        tools=[websearch],
    )
    return agent

# Create agent
agent = create_personal_agent()
logger.info("✅ Personal agent created with MemorySession and web search")

2025-12-13 18:42:53,137 - INFO - ✅ Memory hooks registered with MemorySession
2025-12-13 18:42:53,195 - INFO - Retrieved total of 0 events
2025-12-13 18:42:53,196 - INFO - ✅ Personal agent created with MemorySession and web search


#### Congratulations ! Your agent is ready with the MemoryManager & MemorySession
## Let's test the Agent

In [11]:
# Test conversation with memory
print("=== First Conversation ===")
print(f"User: My name is Amsh and I'm interested in learning about AI.")
print(f"Agent: ", end="")
agent("My name is Amsh and I'm interested in learning about AI.")

2025-12-13 18:43:18,844 - INFO -   -> Storing 1 messages in short-term memory...
2025-12-13 18:43:19,033 - INFO -      ✅ Turn stored successfully with Event ID: None
2025-12-13 18:43:19,033 - INFO - ✅ Stored message with Event ID: 0000001765651398845#a9c62c19, Role: USER
2025-12-13 18:43:19,034 - INFO - Creating Strands MetricsClient


=== First Conversation ===
User: My name is Amsh and I'm interested in learning about AI.
Agent: Hi Amsh! It's great to meet you, and I love your interest in AI! 🤖

I'd be happy to help you learn about artificial intelligence. AI is a fascinating and rapidly evolving field with many exciting applications and concepts. Here are some areas we could explore together:

**Foundational Topics:**
- What is AI and how does it work?
- Machine Learning vs. Deep Learning vs. Neural Networks
- Common AI applications and real-world uses
- The history and evolution of AI

**Practical Areas:**
- Natural Language Processing (NLP)
- Computer Vision
- Generative AI and Large Language Models
- Robotics and Autonomous Systems

**Career & Learning Paths:**
- How to get started with AI learning
- Programming languages and tools for AI
- Educational resources and courses
- Career opportunities in AI

**Current Topics:**
- Latest AI developments and breakthroughs
- Ethical considerations in AI
- AI safety and

2025-12-13 18:43:23,090 - INFO -   -> Storing 1 messages in short-term memory...
2025-12-13 18:43:23,244 - INFO -      ✅ Turn stored successfully with Event ID: None
2025-12-13 18:43:23,245 - INFO - ✅ Stored message with Event ID: 0000001765651403091#448a18b1, Role: ASSISTANT


 AI interests you most right now?

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Hi Amsh! It's great to meet you, and I love your interest in AI! 🤖\n\nI'd be happy to help you learn about artificial intelligence. AI is a fascinating and rapidly evolving field with many exciting applications and concepts. Here are some areas we could explore together:\n\n**Foundational Topics:**\n- What is AI and how does it work?\n- Machine Learning vs. Deep Learning vs. Neural Networks\n- Common AI applications and real-world uses\n- The history and evolution of AI\n\n**Practical Areas:**\n- Natural Language Processing (NLP)\n- Computer Vision\n- Generative AI and Large Language Models\n- Robotics and Autonomous Systems\n\n**Career & Learning Paths:**\n- How to get started with AI learning\n- Programming languages and tools for AI\n- Educational resources and courses\n- Career opportunities in AI\n\n**Current Topics:**\n- Latest AI developments and breakthroughs\n- Ethical considerations in AI\

In [12]:
print(f"User: Can you search for the latest AI trends in 2025?")
print(f"Agent: ", end="")
agent("Can you search for the latest AI trends in 2025?")

2025-12-13 18:43:31,956 - INFO -   -> Storing 1 messages in short-term memory...
2025-12-13 18:43:32,094 - INFO -      ✅ Turn stored successfully with Event ID: None
2025-12-13 18:43:32,095 - INFO - ✅ Stored message with Event ID: 0000001765651411957#c9300661, Role: USER


User: Can you search for the latest AI trends in 2025?
Agent: 
Tool #1: websearch


2025-12-13 18:43:34,342 - INFO - response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=latest%20AI%20trends%202025 200
2025-12-13 18:43:35,500 - INFO - response: https://yandex.com/search/site/?text=latest+AI+trends+2025&web=1&searchid=1843299 200


Great! Here are the latest AI trends in 2025 based on current information:

## **Top AI Trends in 2025:**

### **1. Advanced AI Assistants**
- The AI assistant market is booming with multiple competing platforms
- Popular options include ChatGPT, Claude, and newer models
- Even traditional assistants like Siri are being enhanced with AI capabilities

### **2. Grok 4.1 - xAI's Latest Upgrade**
- Elon Musk's xAI company released **Grok 4.1 Beta** in late November 2025
- Represents a significant upgrade to their flagship chatbot
- Shows the continued competition and rapid innovation in AI models

### **3. Autonomous AI Research Agents**
- **Google introduced the Gemini Deep Research agent** for developers
- These AI agents can autonomously plan, identify information gaps, and navigate the web
- They can produce detailed research reports without human intervention
- This represents a shift toward more independent AI capabilities

### **4. Generative AI for Creative Applications**
- **AI ph

2025-12-13 18:43:40,034 - INFO -   -> Storing 1 messages in short-term memory...


 opportunities in AI for 2025
- Specific applications in your field of interest

2025-12-13 18:43:40,218 - INFO -      ✅ Turn stored successfully with Event ID: None
2025-12-13 18:43:40,219 - INFO - ✅ Stored message with Event ID: 0000001765651420035#790bed23, Role: ASSISTANT


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Great! Here are the latest AI trends in 2025 based on current information:\n\n## **Top AI Trends in 2025:**\n\n### **1. Advanced AI Assistants**\n- The AI assistant market is booming with multiple competing platforms\n- Popular options include ChatGPT, Claude, and newer models\n- Even traditional assistants like Siri are being enhanced with AI capabilities\n\n### **2. Grok 4.1 - xAI's Latest Upgrade**\n- Elon Musk's xAI company released **Grok 4.1 Beta** in late November 2025\n- Represents a significant upgrade to their flagship chatbot\n- Shows the continued competition and rapid innovation in AI models\n\n### **3. Autonomous AI Research Agents**\n- **Google introduced the Gemini Deep Research agent** for developers\n- These AI agents can autonomously plan, identify information gaps, and navigate the web\n- They can produce detailed research reports without human intervention\n- This represents a s

In [13]:
print(f"User: I'm particularly interested in machine learning applications.")
print(f"Agent: ", end="")
agent("I'm particularly interested in machine learning applications.")

2025-12-13 18:43:46,254 - INFO -   -> Storing 1 messages in short-term memory...
2025-12-13 18:43:46,385 - INFO -      ✅ Turn stored successfully with Event ID: None
2025-12-13 18:43:46,385 - INFO - ✅ Stored message with Event ID: 0000001765651426256#258276bf, Role: USER


User: I'm particularly interested in machine learning applications.
Agent: Excellent choice, Amsh! Machine Learning is one of the most practical and impactful areas of AI. Let me search for the latest information on machine learning applications in 2025.
Tool #2: websearch


2025-12-13 18:43:48,499 - INFO -   -> Storing 1 messages in short-term memory...
2025-12-13 18:43:48,665 - INFO -      ✅ Turn stored successfully with Event ID: None
2025-12-13 18:43:48,666 - INFO - ✅ Stored message with Event ID: 0000001765651428501#6a8a0612, Role: ASSISTANT
2025-12-13 18:43:48,764 - INFO - response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=machine%20learning%20applications%202025 200
2025-12-13 18:43:49,558 - INFO - HTTP Request: POST https://html.duckduckgo.com/html/ "HTTP/2 200 OK"


Perfect! Here's what I found about Machine Learning applications in 2025:

## **Key Machine Learning Trends & Applications in 2025:**

### **Market Growth**
- The ML market is **expected to reach $666.16 billion by 2032**
- Rapid growth driven by technological advancements and increasing demand

### **Major Industry Applications:**

**1. Healthcare**
- Diagnostic systems using ML for disease detection
- Personalized treatment recommendations
- Drug discovery and development acceleration

**2. Finance**
- Fraud detection and prevention
- Risk assessment and management
- Algorithmic trading and portfolio optimization
- Credit scoring and loan approval systems

**3. Retail & E-Commerce**
- Personalized product recommendations
- Inventory management and demand forecasting
- Customer behavior prediction
- Price optimization

**4. Decision-Making & Interpretability** ⭐
- Growing focus on **trustworthy AI and explainable ML**
- Discussions about confidence in ML model decisions
- Transparency

2025-12-13 18:43:54,335 - INFO -   -> Storing 1 messages in short-term memory...


 in ML** - what skills and certifications are valuable

What interests you most?

2025-12-13 18:43:54,479 - INFO -      ✅ Turn stored successfully with Event ID: None
2025-12-13 18:43:54,480 - INFO - ✅ Stored message with Event ID: 0000001765651434336#bcca3902, Role: ASSISTANT


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Perfect! Here's what I found about Machine Learning applications in 2025:\n\n## **Key Machine Learning Trends & Applications in 2025:**\n\n### **Market Growth**\n- The ML market is **expected to reach $666.16 billion by 2032**\n- Rapid growth driven by technological advancements and increasing demand\n\n### **Major Industry Applications:**\n\n**1. Healthcare**\n- Diagnostic systems using ML for disease detection\n- Personalized treatment recommendations\n- Drug discovery and development acceleration\n\n**2. Finance**\n- Fraud detection and prevention\n- Risk assessment and management\n- Algorithmic trading and portfolio optimization\n- Credit scoring and loan approval systems\n\n**3. Retail & E-Commerce**\n- Personalized product recommendations\n- Inventory management and demand forecasting\n- Customer behavior prediction\n- Price optimization\n\n**4. Decision-Making & Interpretability** ⭐\n- Growin

## Test Memory Continuity with MemorySessionManager

To test if our memory system is working correctly, we'll create a new instance of the agent and see if it can access the previously stored information using MemorySessionManager:

In [14]:
# Create new agent instance (simulates user returning)
print("=== User Returns - New Session ===")
new_agent = create_personal_agent()

# Test memory continuity
print(f"User: What was my name again?")
print(f"Agent: ", end="")
new_agent("What was my name again?")

print(f"User: Can you search for more information about machine learning?")
print(f"Agent: ", end="")
new_agent("Can you search for more information about machine learning?")

2025-12-13 18:44:11,824 - INFO - ✅ Memory hooks registered with MemorySession
2025-12-13 18:44:11,931 - INFO - Retrieved total of 7 events
2025-12-13 18:44:11,932 - INFO - ✅ Loaded 4 conversation turns using MemorySession
2025-12-13 18:44:11,933 - INFO -   -> Storing 1 messages in short-term memory...


=== User Returns - New Session ===
User: What was my name again?
Agent: 

2025-12-13 18:44:12,068 - INFO -      ✅ Turn stored successfully with Event ID: None
2025-12-13 18:44:12,068 - INFO - ✅ Stored message with Event ID: 0000001765651451934#52008ddb, Role: USER


Your name is **Amsh**! 

2025-12-13 18:44:13,598 - INFO -   -> Storing 1 messages in short-term memory...


...

Perfect! Here's the latest information about **Machine Learning in 2025**, Amsh:

## **Top Machine Learning Applications & Trends in 2025:**

### **Key Application Areas:**

**1. Image Recognition & Computer Vision** 👁️
- ML-enabled computer vision identifies and classifies objects/patterns in images
- Facial recognition technology uses supervised learning
- Increasingly used across industries for automated analysis

**2. Healthcare Applications** 🏥
- Diagnostic systems and disease detection
- Personalized treatment recommendations
- Integration into robotics for precision medical procedures

**3. Finance & Fraud Detection** 💰
- Identifying potential fraud in credit card transactions
- Risk assessment and management
- These applications are proving so effective that organizations prefer to keep them rather than replace them with newer systems

**4. Robotics & Manufacturing** 🤖
- Robots widely used in manufacturing with ML integration
- Increasing adoption in healthcare robotics
- ML ma

2025-12-13 18:44:23,252 - INFO -   -> Storing 1 messages in short-term memory...


 🚀

2025-12-13 18:44:23,410 - INFO -      ✅ Turn stored successfully with Event ID: None
2025-12-13 18:44:23,411 - INFO - ✅ Stored message with Event ID: 0000001765651463253#cf111316, Role: ASSISTANT


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Perfect! Here's the latest information about **Machine Learning in 2025**, Amsh:\n\n## **Top Machine Learning Applications & Trends in 2025:**\n\n### **Key Application Areas:**\n\n**1. Image Recognition & Computer Vision** 👁️\n- ML-enabled computer vision identifies and classifies objects/patterns in images\n- Facial recognition technology uses supervised learning\n- Increasingly used across industries for automated analysis\n\n**2. Healthcare Applications** 🏥\n- Diagnostic systems and disease detection\n- Personalized treatment recommendations\n- Integration into robotics for precision medical procedures\n\n**3. Finance & Fraud Detection** 💰\n- Identifying potential fraud in credit card transactions\n- Risk assessment and management\n- These applications are proving so effective that organizations prefer to keep them rather than replace them with newer systems\n\n**4. Robotics & Manufacturing** 🤖\n

## View Stored Memory using MemorySession

In [15]:
# Check what's stored in memory using MemorySession
print("=== Memory Contents ===")
recent_turns = user_session.get_last_k_turns(k=3) 

for i, turn in enumerate(recent_turns, 1):
    print(f"Turn {i}:")
    for message in turn:
        role = message['role']
        content = message['content']['text'][:100] + "..." if len(message['content']['text']) > 100 else message['content']['text']
        print(f"  {role}: {content}")
    print()

2025-12-13 18:44:42,557 - INFO - Retrieved total of 11 events


=== Memory Contents ===
Turn 1:
  ASSISTANT: Perfect! Here's the latest information about **Machine Learning in 2025**, Amsh:

## **Top Machine L...

Turn 2:
  USER: Can you search for more information about machine learning?
  ASSISTANT: Your name is **Amsh**! 😊

You mentioned that at the beginning of our conversation when you said you ...

Turn 3:
  USER: What was my name again?
  ASSISTANT: Perfect! Here's what I found about Machine Learning applications in 2025:

## **Key Machine Learning...
  ASSISTANT: Excellent choice, Amsh! Machine Learning is one of the most practical and impactful areas of AI. Let...



## Summary

This tutorial showed how to build a personal agent using both MemorySessionManager and MemorySession. You've learned:

- **MemorySessionManager**: High-level manager for memory operations across multiple sessions
- **MemorySession**: Session-specific interface that eliminates repetitive parameter passing. Using MemorySession removes the need to pass actor_id/session_id to every method
- **Type Safety**: Session is bound to specific actor/session at creation time
- **Better Encapsulation**: Session-specific operations are contained within the session object
- **Memory Hooks**: Agent hooks can work with the session-based architecture
- **Conversation Continuity**: Maintaining short-term memory functionality with MemoryManager & MemorySession

### Key Benefits of MemorySession:
1. **Simplified API**: No need to pass actor_id/session_id to every method call
2. **Pre-configured Context**: Session is bound to specific actor/session at creation
3. **Consistent Interface**: All session operations use the same pre-configured context


## Cleanup (Optional)

In [ ]:
# Uncomment to delete memory resource using MemoryManager
# try:
#     memory_manager.delete_memory(memory_id)
#     logger.info(f"✅ Deleted memory: {memory_id}")
# except Exception as e:
#     logger.error(f"Failed to delete memory: {e}")